In [1]:
import pandas as pd
import numpy as np

# Code requirements

- Create deck
- Shuffle deck
- Deal a card
- Save dealer and player hands
- Calculate points
    - Deal with Aces
- Simulate a game
    - Deal with whether the player should hit
    - Deal with whether the dealer needs to hit
- Simulate trials
    - To find expected wins
- Save our data into a dataframe

# Deck

In [2]:
class Deck():
    values = ["A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K"]
    def __init__(self, num_decks=1, values=values):
        # Multiply values by number of suits
        # Then by number of decks
        self.num_decks = num_decks
        self.deck = values * 4 * self.num_decks
        
    def __str__(self):
        return "{} decks, {} cards left".format(self.num_decks, len(self.deck))
    
    def shuffle(self):
        np.random.shuffle(self.deck)
    
    def deal(self, hand=None):
        if hand == None:
            return self.deck.pop(0)
        else:
            hand.append(self.deck.pop(0))

In [3]:
test = Deck()
print (test)
print (test.deck)
print (test.deal())
print (test.deck)
test.shuffle()
print (test.deck)

1 decks, 52 cards left
['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']
A
[2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']
['Q', 'K', 5, 'A', 5, 9, 8, 3, 10, 4, 'J', 8, 3, 2, 7, 9, 4, 5, 'A', 8, 10, 6, 7, 7, 'J', 10, 3, 9, 6, 5, 'J', 10, 'Q', 4, 2, 2, 'K', 6, 2, 4, 7, 8, 3, 'A', 'K', 'K', 'Q', 'Q', 6, 9, 'J']


# Calculate points

In [4]:
def calculate_points(hand):
    points = 0
    num_ace = 0
    
    # Deal with aces
    while "A" in hand:
        hand.remove("A")
        num_ace += 1
    
    # Deal with the rest of the hand
    for i in hand:
        try:
            # If it's a number, add it to points
            points += i
        except:
            # If it's not a number, it's a 10
            points += 10
            
    # deal with the aces
    for i in range(num_ace):
        if points + 11 <= 21:
            points += 11
        else:
            points += 1
    return points

In [5]:
calculate_points([5, 5, "A"])

21

# Simulate game

In [6]:
data_dictionary = pd.read_csv("data_dictionary.csv")
data_dictionary

,Feature,Type,Description
0,num_decks,Int,Number of decks used
1,dealer_open,Int,The card we can see
2,dealer_initial,Int,The dealer's starting points
3,dealer_hit,Binary,"1 - dealer hit, 0 - dealer did not hit"
4,dealer_num_hits,Int,Number of times the dealer hit
5,dealer_final,Int,The dealer's final points
6,dealer_busts,Binary,"1 - dealer busts, 0 - dealer did not bust"
7,player_card_one,Int,Player's first card
8,player_card_two,Int,Player's second card
9,player_initial,Int,The player's starting points


In [44]:
def soft_17(hand):
    if calculate_points(hand.copy()) == 17:
        if "A" in hand:
            hand_copy = hand.copy()
            hand_copy.remove("A")
            if calculate_points(hand_copy.copy()) == 6:
                print (hand)
                return True
    else:
        print (hand)
        return False

In [45]:
soft_17(["A", 6])

['A', 6]


True

In [46]:
soft_17(["A", 3, 3])

['A', 3, 3]


True

In [47]:
soft_17(["A", 7])

['A', 7]


False

In [48]:
def sim_game(num_decks=1, strategy=0):
    game_deck = Deck(num_decks=num_decks)
    game_deck.shuffle()

    dealer_hand = []
    player_hand = []

    # deal 2 cards each
    for i in range(2):
        game_deck.deal(player_hand)
        game_deck.deal(dealer_hand)

    # get their initial points
    # copy so that our hand is not changed
    player_initial = calculate_points(player_hand.copy())
    dealer_initial = calculate_points(dealer_hand.copy())

    # get dealer open card
    dealer_open = dealer_hand[0]

    # get cards from player
    player_card_one = player_hand[0]
    player_card_two = player_hand[1]

    # Change them to numerics
    if dealer_open in ["J", "Q", "K"]:
        dealer_open = 10
    elif dealer_open == "A":
        dealer_open = 1

    if player_card_one in ["J", "Q", "K"]:
        player_card_one = 10
    elif player_card_one == "A":
        player_card_one = 1

    if player_card_two in ["J", "Q", "K"]:
        player_card_two = 10
    elif player_card_two == "A":
        player_card_two = 1

    dealer_hit = 0
    dealer_num_hits = 0
    player_hit = 0
    player_num_hits = 0
    player_busts = 0
    dealer_busts = 0
    dealer_final = calculate_points(dealer_hand.copy())
    player_final = calculate_points(player_hand.copy())


    # if anyone got a blackjack, the game should end
    if player_initial != 21 and dealer_initial != 21:
        # if neither of them got a blackjack game continues
        # the player goes first

        # If player <= 11, hit
        while calculate_points(player_hand.copy()) <= 11:
            player_hit = 1
            player_num_hits += 1
            game_deck.deal(player_hand)

        # If strategy is random, randomize hit for 18 and below
        if strategy == 0:
            while calculate_points(player_hand.copy()) <= 18:
                if np.random.random() < 0.5:
                    player_hit = 1
                    player_num_hits += 1
                    game_deck.deal(player_hand)
        # If strategy is recommended, stand on 17 and above
        else:
            if dealer_open <= 6:
                while calculate_points(player_hand.copy()) <= 16:
                    player_hit = 1
                    player_num_hits += 1
                    game_deck.deal(player_hand)

        # update player's final and busts
        player_final = calculate_points(player_hand.copy())
        player_busts = player_final > 21
        # dealer's turn

        # If player didn't bust
        if not player_busts:
            # If dealer < 17 or soft 17, hit
            while calculate_points(dealer_hand.copy()) < 17 or soft_17(dealer_hand.copy()):
                dealer_hit = 1
                dealer_num_hits += 1
                game_deck.deal(dealer_hand)

        # update dealer's final and busts
        dealer_final = calculate_points(dealer_hand.copy())
        dealer_busts = dealer_final > 21

    player_loses = 0
    draw = 0
    player_wins = 0

    # Check who wins
    if player_initial == 21 and dealer_initial != 21:
        player_wins = 1
    elif dealer_initial == 21 and player_initial != 21:
        player_loses = 1
    elif player_busts:
        player_loses = 1
    elif dealer_busts:
        player_wins = 1
    elif player_final > dealer_final:
        player_wins = 1
    elif player_final < dealer_final:
        player_loses = 1
    elif player_final == dealer_final:
        draw = 1

    # Change hands to strings
    dealer_hand_str = ",".join([str(i) for i in dealer_hand])
    player_hand_str = ",".join([str(i) for i in player_hand])
    
    return np.array([num_decks, dealer_open, dealer_initial, 
                     dealer_hit, dealer_num_hits, dealer_final, 
                     int(dealer_busts), player_card_one, 
                     player_card_two, player_initial, player_hit, 
                     player_num_hits, player_final, int(player_busts), 
                     player_loses, draw, player_wins, strategy, 
                     dealer_hand_str, player_hand_str])

In [49]:
pd.Series(sim_game(), index=data_dictionary.Feature.values)

num_decks                1
dealer_open              2
dealer_initial          12
dealer_hit               0
dealer_num_hits          0
dealer_final            12
dealer_busts             0
player_card_one          3
player_card_two          5
player_initial           8
player_hit               1
player_num_hits          2
player_final            22
player_busts             1
player_loses             1
draw                     0
player_wins              0
strategy                 0
dealer_hand           2,10
player_hand        3,5,4,K
dtype: object

# Generate a dataframe

In [9]:
data_dictionary.Feature.values

array(['num_decks', 'dealer_open', 'dealer_initial', 'dealer_hit',
       'dealer_num_hits', 'dealer_final', 'dealer_busts',
       'player_card_one', 'player_card_two', 'player_initial',
       'player_hit', 'player_num_hits', 'player_final', 'player_busts',
       'player_loses', 'draw', 'player_wins', 'strategy', 'dealer_hand',
       'player_hand'], dtype=object)

In [10]:
def gen_data(num_decks=1, df_size=5000, strategy=0):
    return np.array([sim_game(num_decks=num_decks, strategy=strategy) for _ in range(df_size)])

In [11]:
def gen_df(data):
    tmp = pd.DataFrame(data, columns=data_dictionary.Feature.values)
    tmp[tmp.columns.values[:-2]] = tmp[tmp.columns.values[:-2]].astype(int)
    return tmp

In [12]:
ran = gen_df(gen_data(num_decks=4))

In [13]:
rec = gen_df(gen_data(num_decks=4, strategy=1))

In [14]:
df = pd.concat([ran,rec])

In [15]:
# Combine old files if they are there
try:
    old_df = pd.read_csv("blackjack.csv")
    df = pd.concat([old_df, df])
except:
    pass

In [16]:
# save to file
df.to_csv("blackjack.csv", index=False)

# Testing our code

In [17]:
df = pd.read_csv("blackjack.csv")

In [18]:
df.shape

(50000, 20)

In [19]:
# When player_loses = 1, draw = 0, player_wins = 0
print (df.draw[df.player_loses == 1].value_counts())
print (df.player_wins[df.player_loses == 1].value_counts())

0    27464
Name: draw, dtype: int64
0    27464
Name: player_wins, dtype: int64


In [20]:
# When player_loses = 0, draw = 1, player_wins = 0
print (df.player_loses[df.draw == 1].value_counts())
print (df.player_wins[df.draw == 1].value_counts())

0    3569
Name: player_loses, dtype: int64
0    3569
Name: player_wins, dtype: int64


In [21]:
# When player_loses = 0, draw = 0, player_wins = 1
print (df.player_loses[df.player_wins == 1].value_counts())
print (df.draw[df.player_wins == 1].value_counts())

0    18967
Name: player_loses, dtype: int64
0    18967
Name: draw, dtype: int64


In [22]:
# When dealer_busts = 1, player_loses = 0, draw = 0, player_wins = 1
print (df.player_loses[df.dealer_busts == 1].value_counts())
print (df.draw[df.dealer_busts == 1].value_counts())
print (df.player_wins[df.dealer_busts == 1].value_counts())

0    8806
Name: player_loses, dtype: int64
0    8806
Name: draw, dtype: int64
1    8806
Name: player_wins, dtype: int64


In [23]:
# When player_busts = 1, player_loses = 1, draw = 0, player_wins = 0, dealer_hit = 0, dealer_busts = 0
print (df.player_loses[df.player_busts == 1].value_counts())
print (df.draw[df.player_busts == 1].value_counts())
print (df.player_wins[df.player_busts == 1].value_counts())
print (df.dealer_hit[df.player_busts == 1].value_counts())
print (df.dealer_busts[df.player_busts == 1].value_counts())

1    14974
Name: player_loses, dtype: int64
0    14974
Name: draw, dtype: int64
0    14974
Name: player_wins, dtype: int64
0    14974
Name: dealer_hit, dtype: int64
0    14974
Name: dealer_busts, dtype: int64


In [24]:
# When dealer_busts = 0 and player_busts = 0, if player_final > dealer final, player_loses = 0, draw = 0, player_wins = 1
sub = df[(df.player_busts == 0) & (df.dealer_busts == 0)]
print (sub.player_loses[sub.player_final > sub.dealer_final].value_counts())
print (sub.draw[sub.player_final > sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final > sub.dealer_final].value_counts())

0    10161
Name: player_loses, dtype: int64
0    10161
Name: draw, dtype: int64
1    10161
Name: player_wins, dtype: int64


In [25]:
# When dealer_busts = 0 and player_busts = 0, if player_final = dealer final, player_loses = 0, draw = 1, player_wins = 0
print (sub.player_loses[sub.player_final == sub.dealer_final].value_counts())
print (sub.draw[sub.player_final == sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final == sub.dealer_final].value_counts())

0    3569
Name: player_loses, dtype: int64
1    3569
Name: draw, dtype: int64
0    3569
Name: player_wins, dtype: int64


In [26]:
# When dealer_busts = 0 and player_busts = 0, if player_final < dealer final, player_loses = 1, draw = 0, player_wins = 0
print (sub.player_loses[sub.player_final < sub.dealer_final].value_counts())
print (sub.draw[sub.player_final < sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final < sub.dealer_final].value_counts())

1    12490
Name: player_loses, dtype: int64
0    12490
Name: draw, dtype: int64
0    12490
Name: player_wins, dtype: int64


In [27]:
# When player_final <= 21, player_busts == 0
print (df.player_busts[df.player_final <= 21].value_counts())

0    35026
Name: player_busts, dtype: int64


In [28]:
# When dealer_final <= 21, dealer_busts == 0
print (df.dealer_busts[df.dealer_final <= 21].value_counts())

0    41194
Name: dealer_busts, dtype: int64


In [29]:
# When player_hit == 1, player_num_hits >= 1
print (df.player_num_hits[df.player_hit == 1].value_counts())

1    18256
2     8239
3     1987
4      341
5       34
6        4
Name: player_num_hits, dtype: int64


In [30]:
# When dealer_hit == 1, dealer_num_hits >= 1
print (df.dealer_num_hits[df.dealer_hit == 1].value_counts())

1    13933
2     5236
3      981
4      125
5        6
Name: dealer_num_hits, dtype: int64


In [31]:
df.dealer_hit.value_counts()

0    29719
1    20281
Name: dealer_hit, dtype: int64

In [32]:
df.dealer_num_hits.value_counts()

0    29719
1    13933
2     5236
3      981
4      125
5        6
Name: dealer_num_hits, dtype: int64

In [33]:
df.player_hit.value_counts()

1    28861
0    21139
Name: player_hit, dtype: int64

In [34]:
df.player_num_hits.value_counts()

0    21139
1    18256
2     8239
3     1987
4      341
5       34
6        4
Name: player_num_hits, dtype: int64

In [35]:
df[["dealer_final", "player_final"]][df.strategy == 0].describe()

,dealer_final,player_final
count,25000.000000,25000.000000
mean,17.181840,21.869520
std,4.497553,3.205175
min,4.000000,4.000000
25%,14.000000,20.000000
50%,18.000000,21.000000
75%,20.000000,24.000000
max,26.000000,28.000000


In [36]:
df[["dealer_final", "player_final"]][df.strategy == 1].describe()

,dealer_final,player_final
count,25000.000000,25000.00000
mean,18.995760,17.95296
std,3.995685,3.51962
min,4.000000,4.00000
25%,17.000000,15.00000
50%,19.000000,18.00000
75%,21.000000,20.00000
max,26.000000,26.00000


In [42]:
df[(df["dealer_final"] < 17) & (df["player_initial"] != 21) & (df["player_busts"] != 1)]

,num_decks,dealer_open,dealer_initial,dealer_hit,dealer_num_hits,dealer_final,dealer_busts,player_card_one,player_card_two,player_initial,player_hit,player_num_hits,player_final,player_busts,player_loses,draw,player_wins,strategy,dealer_hand,player_hand
926,4,1,17,1,1,12,0,4,5,9,1,1,19,0,0,0,1,0,"A,6,5","4,5,10"
1033,4,4,15,1,2,13,0,9,4,13,1,1,20,0,0,0,1,0,"4,A,2,6","9,4,7"
1305,4,1,17,1,1,12,0,8,10,18,1,1,20,0,0,0,1,0,"A,6,5","8,Q,2"
1527,4,6,17,1,1,12,0,4,4,8,1,1,19,0,0,0,1,0,"6,A,5","4,4,A"
1762,4,6,17,1,1,14,0,1,9,20,0,0,20,0,0,0,1,0,"6,A,7","A,9"
2210,4,6,17,1,1,16,0,10,3,13,1,1,20,0,0,0,1,0,"6,A,9","K,3,7"
2309,4,6,17,1,1,16,0,10,9,19,0,0,19,0,0,0,1,0,"6,A,9","K,9"
2435,4,1,17,1,1,13,0,10,2,12,1,2,19,0,0,0,1,0,"A,6,6","10,2,6,A"
2478,4,3,14,1,2,12,0,8,3,11,1,1,21,0,0,0,1,0,"3,A,3,5","8,3,10"
2635,4,1,17,1,1,14,0,1,9,20,0,0,20,0,0,0,1,0,"A,6,7","A,9"
